In [3]:
#熟悉單網站多網頁，透過列表將連結內的文章內容都爬下來
import requests
import re
import json
from urllib.parse import urljoin
from bs4 import BeautifulSoup


In [8]:
def crawl_article(url):
    
    author = ''
    title = ''
    date = ''
    
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-content')
    
    # 假如文章有屬性資料 (meta), 從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline')
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
            
    # 取得留言區
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
        
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
        
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
        
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    messages = []
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')  #判斷是推文, 箭頭還是噓文
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')   #判斷留言的人是誰
        push_content = push.find('span', 'push-content').strings   #判斷留言內容
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')  #判斷留言日期時間

        # 整理留言資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
            
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

In [9]:
# 對文章列表送出請求並取得列表主體
PTT_URL = 'https://www.ptt.cc/bbs/TAINAN/index.html'
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')

all_data = []

# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
        
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        article_URL = urljoin(PTT_URL, a_title['href'])
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 呼叫寫好的 function 來對文章進行爬蟲
        parse_data = crawl_article(article_URL)
        
        # 將爬完的資料儲存
        all_data.append(parse_data)
        
        
# 將爬完的資訊存成 json 檔案
with open('parse_data.json', 'w+') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)

Parse [交易] Vitantonio VBL-5 可攜式隨行杯果汁機 - https://www.ptt.cc/bbs/Tainan/M.1584788981.A.0DD.html
Parse [交易]LMC 511(R標29W32L)/NB NY馬拉松長版上衣  - https://www.ptt.cc/bbs/Tainan/M.1584788987.A.671.html
Parse [徵求] 美睫model /半永久眼線（免費） - https://www.ptt.cc/bbs/Tainan/M.1584790076.A.501.html
Parse [交易] N1檢定用書 寵物飲水機 G7咖啡與贈送 - https://www.ptt.cc/bbs/Tainan/M.1584791112.A.13A.html
Parse [自家] 通馬桶/通排水管/清洗水塔/抽化糞池 - https://www.ptt.cc/bbs/Tainan/M.1584791553.A.673.html
Parse [問題] 求推薦東區好吃的三角骨 - https://www.ptt.cc/bbs/Tainan/M.1584791855.A.D9B.html
Parse [交易] 湖池屋咖哩厚切辛口洋芋片2包(接洽中) - https://www.ptt.cc/bbs/Tainan/M.1584792121.A.4EB.html
Parse [徵求] 真空食物包裝機 - https://www.ptt.cc/bbs/Tainan/M.1584792296.A.E0A.html


TypeError: 'NoneType' object is not subscriptable